In [24]:
import numpy as np
import numpy.linalg as nla
np.set_printoptions(suppress=True)

# G&vL 3rd pg. 210
def make_house_vec(x):
    n = x.shape[0]
    dot_1on = x[1:].dot(x[1:])

    # v is our return vector; we hack on v[0]
    v = np.copy(x)
    v[0] = 1.0
    
    if dot_1on < np.finfo(float).eps:
        print('ted')
        beta = 0.0
    else:
        # apply Parlett's formula (G&vL page 210) for safe v_0 = x_0 - norm(X) 
        norm_x= np.sqrt(x[0]**2 + dot_1on)
        print(f'norm:{norm_x}')
        if x[0] <= 0:
            v[0] = x[0] - norm_x
        else:
            v[0] = -dot_1on / (x[0] + norm_x)
        beta = 2 * v[0]**2 / (dot_1on + v[0]**2)
        v = v / v[0]
    return v, beta

In [18]:
def full_house(n, col, v, beta):
    ''' for size n, apply a Householder vector v in the lower right corner of 
        I_n to get a full-sized matrix with a smaller Householder matrix component'''
    full = np.eye(n)
    full[col:, col:] -= beta * np.outer(v,v)
    return full

# G&VL Algo. 5.4.2 with explicit reflections
def house_bidiag_explicit_UV(A):
    m,n = A.shape
    U = np.eye(m)
    Vt = np.eye(n)
    
    for col in range(n):
        v, beta = make_house_vec(A[col:,col])
        print(f'v: {v}, beta: {beta}')
        A[col:,col:] = (np.eye(m-col) - beta * np.outer(v,v)).dot(A[col:,col:])
        Q = full_house(m, col, v, beta)
        U = U.dot(Q)
        
        #if col <= n-2:
        #    # transpose here, reflection for zeros above diagonal in A
        #    # col+1 keeps us off the super diagonal
        #    v,beta = make_house_vec(A[col,col+1:].T)
        #    A[col:,col+1:] = A[col:, col+1:].dot(np.eye(n-(col+1)) - beta * np.outer(v,v))
        #    P = full_house(n, col+1, v, beta)
        #    Vt = P.dot(Vt)
    return U, A, Vt

In [25]:
#A = np.random.rand(5,5)

A = np.arange(1,13.0).reshape(4,3)
A_test = np.copy(A)

print(A)

U, B, Vt = house_bidiag_explicit_UV(A)
print("B:\n", B)
print("U:\n", U)
print("Vt:\n", Vt)

print("should equal input:", np.allclose(U.dot(B).dot(Vt), A_test))

[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]
norm:12.884098726725126
v: [ 1.         -0.33658421 -0.58902237 -0.84146053], beta: 0.9223849474293666
norm:1.0413152017509326
v: [ 1.         -2.08706208 -1.44794203], beta: 0.26837120764369127
ted
v: [1. 0.], beta: 0.0
B:
 [[12.88409873 14.59162988 16.29916104]
 [-0.          1.0413152   2.0826304 ]
 [-0.         -0.         -0.        ]
 [-0.         -0.          0.        ]]
U:
 [[ 0.07761505  0.83305216 -0.54737648  0.01946767]
 [ 0.31046021  0.45123659  0.74434565  0.38203344]
 [ 0.54330537  0.06942101  0.15343813 -0.8224699 ]
 [ 0.77615053 -0.31239456 -0.35040731  0.42096879]]
Vt:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
should equal input: True


In [11]:
print(U@A@Vt)

[[0.17133891 0.98000448 0.23487395 0.94124947 0.00229525 0.41697119]
 [0.27587647 0.65998118 0.0342021  0.84995943 0.045687   0.93001764]
 [0.57259144 0.38638564 0.82930839 0.54302212 0.89839889 0.49886778]
 [0.37913101 0.52795998 0.58414411 0.55812292 0.23001718 0.36430514]
 [0.4839459  0.80058805 0.27197761 0.04454865 0.36717138 0.5489275 ]
 [0.61724049 0.04276734 0.32495907 0.0147594  0.38084114 0.01597929]]
